# Multi-Client Example
*BeamNG.tech / BeamNG.drive*

## Introduction

This example shows how to connect multiple `BeamNGpy` instances to the simulator and have them control different vehicles. For demonstration purposes, both instances are housed in the same Python process, but the same example could be used from an entirely different process or machine as well.

## Scenario

The scenario will be a rather simple one: there are two vehicles on the `smallgrid` map (an infinite flat plane) and one vehicle will randomly drive around, controlled by client A, with another vehicle, controlled by client B, mimicking A's behavior.

## Setup

Contrary to other examples, we will be using two instances of the `BeamNGpy` class representing two clients A and B. In our case, client A will create the scenario containing both vehicles, but client B will later connect and control one of the vehicles while A controls the other. The classes involved in this are:

* `BeamNGpy`: Two instances of this will be used to implement client A and B
* `Scenario`: Client A will create a scenario for both clients to play in
* `Vehicle`: Used to specify and control vehicles involved in the scenario
* `Electrics`: A sensor used to inspect interal systems of the vehicle

The code starts with importing the respective classes:

In [1]:
import random

from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import Electrics

Now we set up our first client who will create the scenario for both clients. Client A will also launch the simulator.

In [2]:
client_a = BeamNGpy('localhost', 64256)
client_a.open()

With the simulator running, we can start setting up our scenario. It will contain two vehicles, the one controlled by Client A being placed in front of the one later controlled by client B.

In [3]:
scenario = Scenario('smallgrid', 'tag')
av_a = Vehicle('vehicleA', model='etk800')
av_b = Vehicle('vehicleB', model='etk800')
scenario.add_vehicle(av_a, pos=(0, -10, 0))
scenario.add_vehicle(av_b)
scenario.make(client_a)

## Running

The scenario is now made, meaning the required files have been generated and can be loaded in the simulator:

In [4]:
client_a.scenario.load(scenario)
client_a.scenario.start()

Now we will set up our second client and connect it to the running simulator. The client will first connect, then query the running scenario, and retrieve currently active vehicles. They will then find the vehicle meant for Client B and connect to it. Note that `client_b` is opened with the flag `launch=False`meaning it will not launch its own BeamNG.tech process.

In [5]:
client_b = BeamNGpy('localhost', 64256)
client_b.open(launch=False)
running_scenario = client_b.scenario.get_current()
print(running_scenario.name)
active_vehicles = client_b.scenario.get_current_vehicles()
bv_a = active_vehicles['vehicleA']
bv_b = active_vehicles['vehicleB']
# B attaches their own sensor to get the current controls of A
bv_a.sensors.attach('electrics', Electrics())
bv_a.connect(client_b)
bv_b.connect(client_b)

tag


Two clients are now connected to the running simulation and both vehicles. What follows is the main loop of the scenario, where Client A sends random steering inputs to their vehicle and Client B checks how A's vehicle is driving using the electrics sensor and sends the same inputs to their vehicle.

In [ ]:
# Focus simulator on second vehicle because it's the more interesting one to see
client_a.vehicles.switch(av_b)

for _ in range(120):
    # Client A sending control inputs to their vehicle connection
    av_a.control(throttle=random.random(), steering=(random.random() * 2 - 1))
    
    # Client B updating the electrics sensor of A's vehicle
    bv_a.sensors.poll()
    
    throttle = bv_a.sensors['electrics']['throttle_input']
    steering = bv_a.sensors['electrics']['steering_input']
    print(throttle, steering)
    bv_b.control(throttle=throttle, steering=steering)
    
    # Client A now advancing the simulation 60 steps
    client_a.control.step(60)

client_b.close()
client_a.close()

0.38597644268602527 0.032619632758827385
0.6140631495373333 0.2443437124722897
0.7631290167443384 -0.1669360816929912
0.7055943520691731 -0.13675721910184505
0.1654727461446155 -0.15493953738021768
0.21997585388210417 -0.16065928065329013
0.9891361957419043 -0.08166718583405448
0.19733242259710138 -0.16611200424041683
0.4174650618368878 -0.004800910519080332
0.856006222035302 0.28371785693240964
0.4898422230077982 0.1442634380685084
0.7748729238759757 -0.15520920337217692
0.9052705018962289 0.22413378561819663
0.47081781810873236 0.16144233830992422
0.7910037471169399 0.04081974662367535
0.8385507582402726 0.25843627080048287
0.13986927910659963 0.1866505260238401
0.1065120624112561 -0.0351320267248174
0.37815844882429595 0.042117492632751025
0.6334357160223022 0.22123064002288223
0.3608837103209406 -0.11555030739540387
0.6715498780379924 -0.23254919830073384
0.27349800511181 -0.16115588911650428
0.003020687150107504 -0.08443388537350907
0.2970691951492431 -0.004043838955167878
0.75301